### Bloc 1.2 – Chargement du CSV Sunspots et premières informations

Nous avons extrait le contenu de `sunspots.zip` dans :

- `data_phase2/sunspots_raw/`

Dans ce bloc, nous allons :

1. détecter automatiquement le (ou les) fichiers `.csv` présents dans `data_phase2/sunspots_raw/` ;
2. imposer qu’il y ait **exactement un** fichier CSV utilisable (sinon erreur explicite) ;
3. charger ce CSV avec `pandas` **sans faire d’hypothèse** sur les noms de colonnes ;
4. afficher et logguer les informations de base :
   - nombre de lignes et de colonnes ;
   - liste des colonnes ;
   - aperçu des premières lignes.

Les décisions sur :
- quelle colonne représente la date ;
- quelle colonne représente l’intensité des taches solaires ;
seront prises dans un bloc suivant (après observation réelle de la structure du fichier),
pour éviter d’introduire des hypothèses fictives.

In [6]:
# Bloc 1.2 – Chargement du CSV Sunspots et premières informations

from pathlib import Path
import pandas as pd

# 1.2.1 – Dossier contenant les CSV extraits
DATA_PHASE2_RAW_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_raw"
print(f"Dossier des données brutes Phase 2 : {DATA_PHASE2_RAW_DIR}")

if not DATA_PHASE2_RAW_DIR.exists():
    raise FileNotFoundError(
        f"Le dossier des données brutes n'existe pas : {DATA_PHASE2_RAW_DIR}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 1.1 (extraction du zip)."
    )

# 1.2.2 – Détection des fichiers CSV présents
csv_files = sorted(DATA_PHASE2_RAW_DIR.rglob("*.csv"))
print("\nFichiers CSV trouvés dans data_phase2/sunspots_raw/ :")
for p in csv_files:
    print(f"  - {p}")

if len(csv_files) == 0:
    raise RuntimeError(
        "Aucun fichier .csv trouvé dans data_phase2/sunspots_raw/ après extraction."
    )
elif len(csv_files) > 1:
    raise RuntimeError(
        "Plusieurs fichiers .csv trouvés dans data_phase2/sunspots_raw/.\n"
        "Pour garantir une configuration claire et reproductible, "
        "la Phase 2 attend exactement un fichier CSV Sunspots.\n"
        f"Fichiers détectés : {[str(p) for p in csv_files]}"
    )

SUNSPOTS_CSV_PATH = csv_files[0]
print(f"\nFichier CSV Sunspots sélectionné : {SUNSPOTS_CSV_PATH}")

# 1.2.3 – Chargement du CSV sans hypothèses sur la structure
df_sunspots = pd.read_csv(SUNSPOTS_CSV_PATH)

# 1.2.4 – Infos de base
n_rows, n_cols = df_sunspots.shape
columns = df_sunspots.columns.tolist()

print("\nRésumé du DataFrame Sunspots :")
print(f"  - Nombre de lignes    : {n_rows}")
print(f"  - Nombre de colonnes  : {n_cols}")
print(f"  - Colonnes            : {columns}")

print("\nAperçu des 5 premières lignes :")
display(df_sunspots.head())

# 1.2.5 – Logging pour audit
log_message(
    "INFO",
    f"CSV Sunspots chargé depuis {SUNSPOTS_CSV_PATH.name} avec shape={df_sunspots.shape}",
    block="BLOC_1.2",
)
log_metric(
    "sunspots_csv_rows",
    int(n_rows),
    extra={"n_cols": int(n_cols), "columns": columns},
)

Dossier des données brutes Phase 2 : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_raw

Fichiers CSV trouvés dans data_phase2/sunspots_raw/ :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_raw\Sunspots.csv

Fichier CSV Sunspots sélectionné : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_raw\Sunspots.csv

Résumé du DataFrame Sunspots :
  - Nombre de lignes    : 3265
  - Nombre de colonnes  : 3
  - Colonnes            : ['Unnamed: 0', 'Date', 'Monthly Mean Total Sunspot Number']

Aperçu des 5 premières lignes :


,Unnamed: 0,Date,Monthly Mean Total Sunspot Number
0,0,1749-01-31,96.7
1,1,1749-02-28,104.3
2,2,1749-03-31,116.7
3,3,1749-04-30,92.8
4,4,1749-05-31,141.7


[STEP=4][INFO][BLOC_1.2] CSV Sunspots chargé depuis Sunspots.csv avec shape=(3265, 3)
[METRIC][sunspots_csv_rows] = 3265 (step=4)


### Bloc 1.3 – Sélection et nettoyage des colonnes (série temporelle Sunspots)

À partir du DataFrame brut `df_sunspots` (3265 lignes, 3 colonnes) :

- `Unnamed: 0` : colonne d’index artificiel (issue du CSV exporté) que nous allons supprimer.
- `Date` : colonne de dates au format texte (`YYYY-MM-DD`), à convertir en `datetime64`.
- `Monthly Mean Total Sunspot Number` : série numérique principale (intensité mensuelle des taches solaires).

Dans ce bloc, nous allons :

1. créer une copie `df_sunspots_clean` pour ne pas modifier l’objet brut ;
2. supprimer la colonne `Unnamed: 0` si elle existe ;
3. convertir `Date` en type datetime (`pd.to_datetime`, `errors="coerce"`) et vérifier qu’il n’y a pas de dates invalides ;
4. convertir `Monthly Mean Total Sunspot Number` en numérique (`pd.to_numeric`, `errors="coerce"`) et vérifier l’absence de valeurs non numériques ;
5. trier les données par `Date` (croissant) et réindexer proprement ;
6. afficher et logguer :
   - la période temporelle couverte (date min / date max) ;
   - le nombre de lignes après nettoyage ;
   - quelques statistiques de base sur la série (min, max, moyenne).

Aucune hypothèse supplémentaire n’est faite sur les données : toutes les décisions (colonnes, types)
sont basées sur la structure réelle observée en Bloc 1.2.

In [7]:
# Bloc 1.3 – Sélection et nettoyage des colonnes

import pandas as pd

# 1.3.1 – Copie de travail
df_sunspots_clean = df_sunspots.copy()

print("Colonnes initiales :", df_sunspots_clean.columns.tolist())

# 1.3.2 – Suppression de la colonne d'index artificiel si présente
if "Unnamed: 0" in df_sunspots_clean.columns:
    df_sunspots_clean = df_sunspots_clean.drop(columns=["Unnamed: 0"])
    print("Colonne 'Unnamed: 0' supprimée.")
else:
    print("Colonne 'Unnamed: 0' absente, aucune suppression nécessaire.")

print("Colonnes après nettoyage initial :", df_sunspots_clean.columns.tolist())

# 1.3.3 – Conversion de la colonne Date en datetime
if "Date" not in df_sunspots_clean.columns:
    raise KeyError("La colonne 'Date' est absente du DataFrame après nettoyage.")

df_sunspots_clean["Date"] = pd.to_datetime(
    df_sunspots_clean["Date"], errors="coerce"
)

nb_nat = df_sunspots_clean["Date"].isna().sum()
if nb_nat > 0:
    raise RuntimeError(
        f"{nb_nat} valeur(s) de date invalide(s) détectée(s) après conversion. "
        "Veuillez vérifier le fichier CSV Sunspots."
    )

# 1.3.4 – Conversion de la colonne d'intensité en numérique
value_col = "Monthly Mean Total Sunspot Number"
if value_col not in df_sunspots_clean.columns:
    raise KeyError(
        f"La colonne '{value_col}' est absente du DataFrame après nettoyage."
    )

df_sunspots_clean[value_col] = pd.to_numeric(
    df_sunspots_clean[value_col], errors="coerce"
)

nb_nan_val = df_sunspots_clean[value_col].isna().sum()
if nb_nan_val > 0:
    raise RuntimeError(
        f"{nb_nan_val} valeur(s) non numérique(s) détectée(s) dans '{value_col}' "
        "après conversion. Veuillez vérifier le fichier CSV Sunspots."
    )

# 1.3.5 – Tri par date et réindexation
df_sunspots_clean = df_sunspots_clean.sort_values("Date").reset_index(drop=True)

n_rows_clean, n_cols_clean = df_sunspots_clean.shape
date_min = df_sunspots_clean["Date"].min()
date_max = df_sunspots_clean["Date"].max()

val_min = df_sunspots_clean[value_col].min()
val_max = df_sunspots_clean[value_col].max()
val_mean = df_sunspots_clean[value_col].mean()

print("\nRésumé après nettoyage :")
print(f"  - Nombre de lignes    : {n_rows_clean}")
print(f"  - Nombre de colonnes  : {n_cols_clean}")
print(f"  - Période temporelle  : {date_min.date()} -> {date_max.date()}")
print(f"  - {value_col} min     : {val_min}")
print(f"  - {value_col} max     : {val_max}")
print(f"  - {value_col} moyenne : {val_mean}")

print("\nAperçu des 5 premières lignes après nettoyage :")
display(df_sunspots_clean.head())

# 1.3.6 – Logging pour audit
log_message(
    "INFO",
    (
        f"Série Sunspots nettoyée : n={n_rows_clean}, "
        f"période={date_min.date()}->{date_max.date()}, "
        f"{value_col} min={val_min}, max={val_max}, mean={val_mean}"
    ),
    block="BLOC_1.3",
)
log_metric(
    "sunspots_clean_rows",
    int(n_rows_clean),
    extra={
        "n_cols": int(n_cols_clean),
        "date_min": str(date_min.date()),
        "date_max": str(date_max.date()),
        "value_col": value_col,
        "val_min": float(val_min),
        "val_max": float(val_max),
        "val_mean": float(val_mean),
    },
)

Colonnes initiales : ['Unnamed: 0', 'Date', 'Monthly Mean Total Sunspot Number']
Colonne 'Unnamed: 0' supprimée.
Colonnes après nettoyage initial : ['Date', 'Monthly Mean Total Sunspot Number']

Résumé après nettoyage :
  - Nombre de lignes    : 3265
  - Nombre de colonnes  : 2
  - Période temporelle  : 1749-01-31 -> 2021-01-31
  - Monthly Mean Total Sunspot Number min     : 0.0
  - Monthly Mean Total Sunspot Number max     : 398.2
  - Monthly Mean Total Sunspot Number moyenne : 81.77877488514548

Aperçu des 5 premières lignes après nettoyage :


,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


[STEP=5][INFO][BLOC_1.3] Série Sunspots nettoyée : n=3265, période=1749-01-31->2021-01-31, Monthly Mean Total Sunspot Number min=0.0, max=398.2, mean=81.77877488514548
[METRIC][sunspots_clean_rows] = 3265 (step=5)


### Synthèse du Bloc 1 – Données & traçabilité

Dans ce Bloc 1, nous avons mis en place un flux de données **reproductible** et **auditables** :

1. **Source des données**
   - Fichier `sunspots.zip` localisé dans le dossier du projet Phase 2 :
     `SunspotPhase2Tlog/sunspots.zip`
   - Aucun accès réseau à Kaggle n’est requis : le zip est supposé identique à celui déjà utilisé
     en Phase 1.

2. **Extraction contrôlée**
   - Contenu du zip inspecté sans hypothèses préalables.
   - Un seul fichier CSV détecté : [Sunspots.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/CLEANvs%20NOCLEAN/Clean/v0.5/pipelines_Tlog_MUTD_v0.5_multi_domain/Pipeline_Tlog_V0.1_Sunspots_En/data/sunspots_raw/Sunspots.csv:0:0-0:0).
   - Extraction vers un sous-dossier dédié :
     `data_phase2/sunspots_raw/Sunspots.csv`
   - Toutes les écritures restent à l’intérieur de `SunspotPhase2Tlog`.

3. **Chargement et nettoyage**
   - Chargement du CSV réel (3265 lignes, 3 colonnes).
   - Suppression de la colonne d’index artificiel `Unnamed: 0`.
   - Conservation de deux colonnes :
     - `Date` (convertie en type datetime, aucune date invalide détectée).
     - `Monthly Mean Total Sunspot Number` (convertie en numérique, aucune valeur non numérique détectée).
   - Tri par date et réindexation propre.
   - Série finale `df_sunspots_clean` :
     - Nombre de lignes : **3265**
     - Période temporelle : **1749-01-31 → 2021-01-31**
     - Valeurs de `Monthly Mean Total Sunspot Number` :
       - min = **0.0**
       - max = **398.2**
       - moyenne ≈ **81.78**

4. **Audit**
   - Les étapes clés ont été logguées via `log_message` et `log_metric` dans :
     - `logs/runs/<RUN_ID>/run_log.txt`
     - `logs/runs/<RUN_ID>/metrics.jsonl`

Ce Bloc 1 fournit une base de données Sunspots propre et traçable pour la Phase 2,
sans chemins inventés ni dépendance à l’environnement de la Phase 1.

### Bloc 1.4 – Export d'un CSV Sunspots nettoyé (persistant)

Pour éviter de dépendre uniquement de l'état en mémoire (DataFrame), nous créons maintenant
un fichier **CSV nettoyé** qui servira de base logique pour la Phase 2.

- Source : `df_sunspots_clean` (2 colonnes : `Date`, `Monthly Mean Total Sunspot Number`).
- Destination : `data_phase2/sunspots_clean/Sunspots_clean.csv`

Le fichier brut [data_phase2/sunspots_raw/Sunspots.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_raw/Sunspots.csv:0:0-0:0) est conservé tel quel comme archive
fidèle du zip original (Phase 2 ne le modifie pas).

In [8]:
# Bloc 1.4 – Export d'un CSV Sunspots nettoyé (persistant)

from pathlib import Path

# 1.4.1 – Vérification que df_sunspots_clean existe
try:
    df_sunspots_clean
except NameError:
    raise RuntimeError(
        "df_sunspots_clean n'est pas défini. "
        "Assurez-vous d'avoir exécuté les blocs 1.2 et 1.3 avant ce bloc."
    )

# 1.4.2 – Dossier cible pour les données nettoyées
DATA_PHASE2_CLEAN_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_clean"
DATA_PHASE2_CLEAN_DIR.mkdir(parents=True, exist_ok=True)

SUNSPOTS_CLEAN_CSV_PATH = DATA_PHASE2_CLEAN_DIR / "Sunspots_clean.csv"

# 1.4.3 – Écriture du CSV nettoyé (sans index)
df_sunspots_clean.to_csv(SUNSPOTS_CLEAN_CSV_PATH, index=False)

print("CSV Sunspots nettoyé écrit à l'emplacement :")
print(f"  - {SUNSPOTS_CLEAN_CSV_PATH}")

# 1.4.4 – Logging pour audit
log_message(
    "INFO",
    f"CSV Sunspots nettoyé sauvegardé dans {SUNSPOTS_CLEAN_CSV_PATH.name}",
    block="BLOC_1.4",
)
log_metric(
    "sunspots_clean_csv_written",
    True,
    extra={"path": str(SUNSPOTS_CLEAN_CSV_PATH)},
)

CSV Sunspots nettoyé écrit à l'emplacement :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_clean\Sunspots_clean.csv
[STEP=6][INFO][BLOC_1.4] CSV Sunspots nettoyé sauvegardé dans Sunspots_clean.csv
[METRIC][sunspots_clean_csv_written] = True (step=6)


### Bloc 1.5 – Source officielle des données Sunspots pour la Phase 2

À partir de maintenant, la source **officielle** des données Sunspots pour la Phase 2 est :

- [data_phase2/sunspots_clean/Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0)

Ce fichier :

- est dérivé du CSV brut [data_phase2/sunspots_raw/Sunspots.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_raw/Sunspots.csv:0:0-0:0) (copie fidèle du zip) ;
- contient uniquement deux colonnes nettoyées :
  - `Date`
  - `Monthly Mean Total Sunspot Number`
- est destiné à être **rechargé depuis disque** à chaque exécution du notebook,
  pour éviter de dépendre uniquement de l’état en mémoire (RAM).

Les blocs suivants (prétraitements, estimation de `d`, T_log, baselines, etc.) doivent
utiliser ce fichier nettoyé comme point de départ.

In [9]:
# Bloc 1.5 – Rechargement standard de Sunspots_clean.csv

import pandas as pd
from pathlib import Path

DATA_PHASE2_CLEAN_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_clean"
SUNSPOTS_CLEAN_CSV_PATH = DATA_PHASE2_CLEAN_DIR / "Sunspots_clean.csv"

print(f"Fichier CSV Sunspots (clean) attendu : {SUNSPOTS_CLEAN_CSV_PATH}")

if not SUNSPOTS_CLEAN_CSV_PATH.exists():
    raise FileNotFoundError(
        f"Sunspots_clean.csv est introuvable à l'emplacement : {SUNSPOTS_CLEAN_CSV_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 1.4 (export du CSV nettoyé)."
    )

df_sunspots_clean = pd.read_csv(SUNSPOTS_CLEAN_CSV_PATH, parse_dates=["Date"])

print("\ndf_sunspots_clean rechargé depuis disque.")
print(df_sunspots_clean.head())

log_message(
    "INFO",
    f"df_sunspots_clean rechargé depuis {SUNSPOTS_CLEAN_CSV_PATH.name} "
    f"avec shape={df_sunspots_clean.shape}",
    block="BLOC_1.5",
)
log_metric(
    "sunspots_clean_reloaded_rows",
    int(df_sunspots_clean.shape[0]),
    extra={"n_cols": int(df_sunspots_clean.shape[1])},
)

Fichier CSV Sunspots (clean) attendu : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_clean\Sunspots_clean.csv

df_sunspots_clean rechargé depuis disque.
        Date  Monthly Mean Total Sunspot Number
0 1749-01-31                               96.7
1 1749-02-28                              104.3
2 1749-03-31                              116.7
3 1749-04-30                               92.8
4 1749-05-31                              141.7
[STEP=7][INFO][BLOC_1.5] df_sunspots_clean rechargé depuis Sunspots_clean.csv avec shape=(3265, 2)
[METRIC][sunspots_clean_reloaded_rows] = 3265 (step=7)


## Bloc 2 – Prétraitements & fenêtres

L’objectif de ce bloc est de préparer la série temporelle Sunspots pour l’analyse
(mesure de `d`, calcul de T_log, baselines, etc.) sans introduire de biais cachés.

Principes méthodologiques :

- **Raw-first** : la référence reste la série mensuelle brute nettoyée
  ([Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0) : `Date`, `Monthly Mean Total Sunspot Number`).
- Toute opération de **prétraitement** (lissage, detrending, normalisation, découpe en fenêtres)
  doit être :
  - explicite,
  - motivée,
  - réversible (idéalement, on garde une copie des données avant/après).

Plan du Bloc 2 (à construire progressivement) :

- **Bloc 2.0** – Construction d’une série temporelle de base :
  - `Date` comme index temporel ;
  - vérification de l’ordre, des doublons et de la fréquence.
- **Bloc 2.1** – Vérification des valeurs manquantes / anomalies simples.
- **Bloc 2.2** – Décisions sur d’éventuels prétraitements (lissage, normalisation, etc.) et sur la
  construction de fenêtres temporelles (taille, recouvrement), avec justification.

Dans ce qui suit, nous commençons par le **Bloc 2.0** : construire un objet de série temporelle
de base, sans transformation du signal (pas de lissage, pas de filtrage).